In [1]:
import findspark
findspark.find()
findspark.init(edit_profile=True)
import pyspark
sc = pyspark.SparkContext()

In [64]:
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import math
import datetime
from random import randint
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [68]:
#Read in the data form txt file and preprocess to standard format
text_file = open("EOWL_words.txt", "r")
lines = text_file.readlines()
text_file.close()
lines = [i[0:-1] for i in lines]

In [75]:
# wlist = sc.textFile("EOWL_words.txt")
wlist = sc.parallelize(lines, 20)

#Return RDD in the form (SortedLetterSequence, NumberOfValidAnagrams)
sequence_num_RDD = wlist.map(lambda word: (''.join(sorted(list(word))),1)).reduceByKey(lambda x,y: x+y)

#Return RDD in the form (SortedLetterSequence1, [Word1a, Word2a, ...])
sequence_words_RDD = wlist.map(lambda word: (''.join(sorted(list(word))),''+ word)).groupByKey().map(lambda x : (x[0], list(x[1])))

#Join the two RDDs above and map to designated format
result = sequence_num_RDD.join(sequence_words_RDD).map(lambda x: (x[0], x[1][0], x[1][1]))

#Extract and print the line from the RDD above with the largest number of valid anagrams.
result = result.sortBy(lambda x: x[1], False)
result.first()

('aelst',
 11,
 ['least',
  'leats',
  'salet',
  'slate',
  'stale',
  'steal',
  'stela',
  'taels',
  'tales',
  'teals',
  'tesla'])

In [71]:
wlist = sc.parallelize(lines, 20)
sequence_words_RDD = wlist.map(lambda word: (''.join(sorted(list(word))),word)).groupByKey().map(lambda x : (x[0], list(x[1])))
result = sequence_words_RDD.map(lambda x: (x[0], len(x[1]), x[1]))

result = result.sortBy(lambda x: x[1], False)

result.first()

('aeprs',
 11,
 ['asper',
  'pares',
  'parse',
  'pears',
  'prase',
  'presa',
  'rapes',
  'reaps',
  'spaer',
  'spare',
  'spear'])